<a href="https://colab.research.google.com/github/jihyunjeongme/data-analysis-kaggle/blob/master/Kaggle_Bike_sharing_demand(Random_Forest).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 노트북 안에 그래프를 그리기 위해
%matplotlib inline

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

In [2]:
# test, traing set 데이터 불러오기

from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
cd /content/gdrive/My Drive/Colab Notebooks/dataanalysis/kaggle/bike-sharing-demand/

/content/gdrive/My Drive/Colab Notebooks/dataanalysis/kaggle/bike-sharing-demand


In [10]:
train = pd.read_csv("train.csv", encoding='utf-8')
train.shape

(10886, 12)

In [11]:

test = pd.read_csv("test.csv", encoding='utf-8')
test.shape

(6493, 9)

In [12]:
train['date'] = pd.to_datetime(train['datetime'])
train.shape

(10886, 13)

In [13]:
train["year"] = train["date"].dt.year
train["month"] = train["date"].dt.month
train["day"] = train["date"].dt.day
train["hour"] = train["date"].dt.hour
train["minute"] = train["date"].dt.minute
train["second"] = train["date"].dt.second
train["dayofweek"] = train["date"].dt.dayofweek
train.shape

(10886, 20)

In [14]:
test['date'] = pd.to_datetime(test['datetime'])
test.shape

(6493, 10)

In [15]:
test["year"] = test["date"].dt.year
test["month"] = test["date"].dt.month
test["day"] = test["date"].dt.day
test["hour"] = test["date"].dt.hour
test["minute"] = test["date"].dt.minute
test["second"] = test["date"].dt.second
test["dayofweek"] = test["date"].dt.dayofweek
test.shape

(6493, 17)